In [1]:
import sys
sys.path.append(r'C:\Users\maxfu\OneDrive\Documentos\Master\Maximaster\Year 2 - Period 1\Project\Double Pendulum\Double Pendulum')
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scinet import *
import scinet.pendulum as edo
from sklearn.feature_selection import mutual_info_regression

Instructions for updating:
non-resource variables are not supported in the long term


### Helper functions for plots

In [8]:
def derivatives(y,t,L1,L2,m1,m2):
    
    g=9.81
    Theta_1,Theta_1_dot,Theta_2,Theta_2_dot = y
    
    C1=np.cos(Theta_1-Theta_2)
    C2=np.sin(Theta_1-Theta_2)
    
    # The first derivative of Theta is named Z, and so the second derivative is named Z_dot
    
    Z_1=Theta_1_dot
    Z_2=Theta_2_dot
    Z_1_dot=(m2*g*np.sin(Theta_2)*C1-m2*C2*(L1*Z_1**2*C1+L2*Z_2**2)-(m1+m2)*g*np.sin(Theta_1))/(L1*(m1+m2*C2**2))
    Z_2_dot=((m1+m2)*(L1*Z_1**2*C2-g*np.sin(Theta_2)+g*np.sin(Theta_1)*C1)+m2*L2*Z_2**2*C2*C1)/(L2*(m1+m2*C2**2))
    
    return Z_1,Z_1_dot,Z_2,Z_2_dot

def gen_input(Th1_i,Th2_i,L1,L2,m1,m2, tt_predicted):
    tt_in = np.linspace(0, 5, 50)
    sol_i = np.array([Th1_i,0,Th2_i,0])
    sol = np.array([odeint(derivatives, sol_i, t_sample, args=(L1,L2,m1,m2)) for _ in tt_predicted])
    Th1,Th2=sol[:,0],sol[:,2]
    sol_res = np.reshape(tt_predicted, (-1, 1))
#   out = in2 #dummy filler
    return [Th1,Th2, sol_res, out]

In [9]:
blue_color='#000cff'
orange_color='#ff7700'

In [10]:
def pendulum_prediction(net, b, kappa):    
    tt_given = np.linspace(0, 10, 250)
    tt_predicted = np.linspace(0, 10, 250)
    sol_given_i = np.array([Th1_i,0,Th2_i,0])
    sol_given = odeint(derivatives, sol_given_i, tt_given, args=(L1,L2,m1,m2))
    Th1_given,Th2_given=sol_given[:,0],sol_given[:,2]
    sol_pred = net.run(gen_input(Th1_i,Th2_i,1,1,1,1, tt_predicted), net.output).ravel()
    Th1_pred,Th2_pred=sol_pred[:,0],sol_pred[:,1]
    
    fig = plt.figure(figsize=(3.4, 2.1))
    ax = fig.add_subplot(111)
    ax.plot(Th1_given, Th2_given, color=orange_color, label='True time evolution')
    ax.plot(Th1_pred, Th2_pred, '--', color=blue_color, label='Predicted time evolution')
    ax.set_xlabel(r'$\Theta_{1}$')
    ax.set_ylabel(r'$\Theta_{2}$')
    handles, labels = ax.get_legend_handles_labels()
    lgd=ax.legend(handles, labels,loc='upper center', bbox_to_anchor=(0.6, 1.3), shadow=True, ncol=1)
    fig.tight_layout()
    return fig

In [11]:
def osc_representation_plot(net, b_range, kappa_range, step_num=100, eval_time=7.5):
    bb = np.linspace(*b_range, num=step_num)
    kk = np.linspace(*kappa_range, num=step_num)
    B, K = np.meshgrid(bb, kk)
    out = np.array([net.run(gen_input(1, 0, b, kappa, [eval_time]), net.mu)[0] for b, kappa in zip(np.ravel(B), np.ravel(K))])
    fig = plt.figure(figsize=(net.latent_size*3.9, 2.1))
    for i in range(net.latent_size):
        zs = out[:, i]
        ax = fig.add_subplot('1{}{}'.format(net.latent_size, i + 1), projection='3d')
        Z = np.reshape(zs, B.shape)
        surf = ax.plot_surface(B, K, Z, rstride=1, cstride=1, cmap=cm.inferno, linewidth=0)
        ax.set_xlabel(r'$b$ [$kg/s$]')
        ax.set_ylabel(r'$\kappa$ [$kg/s^2$]')
        ax.set_zlabel('Latent activation {}'.format(i + 1))
        if (i==2):
            ax.set_zlim(-1,1) #Fix the scale for the third plot, where the activation is close to zero
        ax.set_zticks([-1,-0.5,0,0.5,1])
    fig.tight_layout()
    return fig

## Load pre-trained model

### Parameters
- `latent_size: 2` 
- `input_size: 50`
- `input2_size: 1`
- `output_size: 1`
- `encoder_num_units: [500, 100]`
- other parameters: default values
### Data
- Only kappa and b are varied (in the default intervals), A_0 and delta_0 are fixed
- `t_sample: np.linspace(0, 5, 50)` (fed into the network)
- `t_meas_interval: (0, 10)` (time interval in which prediction times lie)
- training data: 95000 samples
- validation data: 5000 samples

### Training
- `epoch_num: 1000`, `batch_size: 512`, `learning_rate: 1e-3`, `beta: 1e-3`

In [12]:
net_2_latent = nn.Network.from_saved('oscillator')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\maxfu\\OneDrive\\Documentos\\Master\\Maximaster\\Year 2 - Period 1\\Project\\Double pendulum\\Double Pendulum\\tf_save/oscillator.pkl'

In [ ]:
pendulum_prediction(net_2_latent, 0.5, 5.);

In [ ]:
%matplotlib tk
osc_representation_plot(net_2_latent, [0.5, 1], [5, 10]);

## Load pre-trained model

### Parameters
- `latent_size: 3` 
- Rest as for `net_2_latent`

In [ ]:
net_3_latent = nn.Network.from_saved('oscillator_3_latent')

In [ ]:
pendulum_prediction(net_3_latent, 0.5, 5.);

In [ ]:
%matplotlib tk
osc_representation_plot(net_3_latent, [0.5, 1], [5, 10]);

## Calculate L2 norm of error

In [ ]:
data, states, params = edo.oscillator_data(50000, t_meas_interval=(0, 10))

In [ ]:
np.sqrt(net_3_latent.run(data, net_3_latent.recon_loss))

## Calculate mutual information between latent neurons

In [ ]:
data, states, params = edo.oscillator_data(200000, t_meas_interval=(0, 10))

In [ ]:
# Calculate the mutual information between two latent neurons
def mi(net, data, latent_neuron_1, latent_neuron_2):
    mu = net_3_latent.run(data, net_3_latent.mu)
    mi_estimate = mutual_info_regression(mu[:,latent_neuron_1].reshape(-1, 1), mu[:,latent_neuron_2], discrete_features = False)
    return mi_estimate[0]

In [ ]:
mi(net_3_latent, data, 0, 0)

In [ ]:
mi(net_3_latent, data, 0, 1)